In [1]:
# Parameters
cpu = 8
group_name = "Astro-TE_NN"
mem_gb = 10


In [2]:
import pandas as pd
import subprocess
import numpy as np
import pyBigWig
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
#group_name = 'DG_Glut'

In [4]:
home_dir = '/home/qzeng_salk_edu/project/241018_pre_ml'
ct = group_name
dar_dir = '/data/combined_DARs_redo'
abc_dir = '/data/celltype_age_RPM_files'
_type = 'up_peaks'

In [5]:
dar_ct_dict = {'L23_IT_CTX_Glut':'L2-3_IT_CTX_Glut'}
bw_ct_dict = {'L23_IT_CTX_Glut':'L2_3_IT_CTX_Glut'}
if ct in bw_ct_dict.keys():
    dar_ct = dar_ct_dict[ct]
    bw_ct = bw_ct_dict[ct]
else:
    dar_ct = ct
    bw_ct = ct

In [6]:
atac_df  = pd.read_csv(f'{dar_dir}/diff_peaks_{dar_ct}_2vs18.csv', sep=',', index_col=0)

In [7]:
# add a chrom, start, end based on the index 
atac_df['chrom'] = atac_df.index.str.split(':').str[0]
atac_df['start'] = atac_df.index.str.split(':').str[1].str.split('-').str[0].astype(int)
atac_df['end'] = atac_df.index.str.split(':').str[1].str.split('-').str[1].astype(int)
atac_df

,log2(fold_change),p-value,adjusted p-value,chrom,start,end
feature name,,,,,,
chr5:109558426-109558927,1.701923,0.000000e+00,0.000000e+00,chr5,109558426,109558927
chr7:26783186-26783687,3.733156,0.000000e+00,0.000000e+00,chr7,26783186,26783687
chr5:109557248-109557749,1.024222,1.253381e-251,3.117159e-248,chr5,109557248,109557749
chr7:47651736-47652237,4.687733,1.614651e-204,3.011729e-201,chr7,47651736,47652237
chr5:109556731-109557232,1.251654,3.179016e-200,4.743727e-197,chr5,109556731,109557232
...,...,...,...,...,...,...
chr17:6961171-6961672,0.252943,7.581730e-01,7.585797e-01,chr17,6961171,6961672
chr10:79716949-79717450,0.265331,7.663166e-01,7.666249e-01,chr10,79716949,79717450
chr13:52047390-52047891,0.269548,7.710420e-01,7.712487e-01,chr13,52047390,52047891


In [8]:
up_atac_df = pd.read_csv(f'{dar_dir}/up_peaks_{dar_ct}.bed', sep='\t', header = None)
#add index like in the atac_df
up_atac_df.index = up_atac_df[0] + ':' + up_atac_df[1].astype(str) + '-' + up_atac_df[2].astype(str)
# add the log(fold_change) baes on the info in atac_df
up_atac_df['log2FoldChange'] = atac_df['log2(fold_change)']
up_atac_df.head()

,0,1,2,log2FoldChange
chr5:109558426-109558927,chr5,109558426,109558927,1.701923
chr7:26783186-26783687,chr7,26783186,26783687,3.733156
chr5:109557248-109557749,chr5,109557248,109557749,1.024222
chr7:47651736-47652237,chr7,47651736,47652237,4.687733
chr5:109556731-109557232,chr5,109556731,109557232,1.251654


In [9]:
# write the same for down_peaks
down_atac_df = pd.read_csv(f'{dar_dir}/down_peaks_{dar_ct}.bed', sep='\t', header = None)
down_atac_df.index = down_atac_df[0] + ':' + down_atac_df[1].astype(str) + '-' + down_atac_df[2].astype(str)
down_atac_df['log2FoldChange'] = atac_df['log2(fold_change)']
down_atac_df.head()

,0,1,2,log2FoldChange
chr13:23571010-23571511,chr13,23571010,23571511,-1.714639
chr13:23562214-23562715,chr13,23562214,23562715,-1.787490
chr13:23574072-23574573,chr13,23574072,23574573,-1.909879
chr7:44590526-44591027,chr7,44590526,44591027,-1.240907
chr13:23555969-23556470,chr13,23555969,23556470,-1.999283


In [10]:
# combina the down and up
atac_df = pd.concat([up_atac_df, down_atac_df])
# save atac_df to bed
atac_df.to_csv(f'{ct}.aDAR.bed', sep='\t', header=False, index=False)

In [11]:
# use bedtools intersect to find the genes that overlap with the DMRs
command = f"bedtools intersect -a {ct}.aDAR.bed -b {home_dir}/mm10_gene_2kb.bed -wa -wb > {ct}.aDAR_gene.bed"
subprocess.run(command, shell=True)

CompletedProcess(args='bedtools intersect -a Astro-TE_NN.aDAR.bed -b /home/qzeng_salk_edu/project/241018_pre_ml/mm10_gene_2kb.bed -wa -wb > Astro-TE_NN.aDAR_gene.bed', returncode=0)

In [12]:
subprocess.run(f"rm {ct}.aDAR.bed", shell=True)

CompletedProcess(args='rm Astro-TE_NN.aDAR.bed', returncode=0)

In [13]:
dar_gene_df = pd.read_csv(f'{ct}.aDAR_gene.bed', sep='\t', header=None)
dar_gene_df.columns = ['chr','start','end','log2(old/young)','gene_chr','gene_start','gene_end','gene_id','gene_strand','gene_name','gene_type']
dar_gene_df.head()

,chr,start,end,log2(old/young),gene_chr,gene_start,gene_end,gene_id,gene_strand,gene_name,gene_type
0,chr5,109558426,109558927,1.701923,chr5,109552711,109560993,ENSMUSG00000033467.8,-,Crlf2,protein_coding
1,chr5,109557248,109557749,1.024222,chr5,109552711,109560993,ENSMUSG00000033467.8,-,Crlf2,protein_coding
2,chr5,109556731,109557232,1.251654,chr5,109552711,109560993,ENSMUSG00000033467.8,-,Crlf2,protein_coding
3,chr5,109557903,109558404,1.598321,chr5,109552711,109560993,ENSMUSG00000033467.8,-,Crlf2,protein_coding
4,chr5,109167108,109167609,5.082705,chr5,109154068,109194107,ENSMUSG00000091635.1,-,Vmn2r13,protein_coding


In [14]:
atac_rmp = pd.read_csv(f"{abc_dir}/{ct}_RPM.txt", sep = '\t')
# add three coolumns chr, start and end based on ethe index 
atac_rmp['chr'] = atac_rmp.index.str.split(':').str[0]
atac_rmp['start'] = atac_rmp.index.str.split(':').str[1].str.split('-').str[0].astype(int)
atac_rmp['end'] = atac_rmp.index.str.split(':').str[1].str.split('-').str[1].astype(int)
atac_rmp

,Astro-TE NN:2mo,Astro-TE NN:9mo,Astro-TE NN:18mo,chr,start,end
chr1:3399698-3400199,1.935104,1.425978,1.328092,chr1,3399698,3400199
chr1:3400380-3400881,0.408325,0.206821,0.426887,chr1,3400380,3400881
chr1:3671537-3672038,6.811330,7.347597,8.632596,chr1,3671537,3672038
chr1:3670856-3671357,2.272416,2.481855,2.739189,chr1,3670856,3671357
chr1:3672169-3672670,0.822567,1.055877,1.102790,chr1,3672169,3672670
...,...,...,...,...,...,...
chrY:90804895-90805396,2.799096,5.029022,3.960559,chrY,90804895,90805396
chrY:90807492-90807993,7.610226,9.840337,7.861829,chrY,90807492,90807993
chrY:90806986-90807487,1.751654,2.547167,1.529677,chrY,90806986,90807487
chrY:90808570-90809071,6.260979,9.862108,8.846040,chrY,90808570,90809071


In [15]:
dar_gene_df.index = dar_gene_df['chr'] + ':' + dar_gene_df['start'].astype(str) + '-' + dar_gene_df['end'].astype(str)

In [16]:
shared_index = dar_gene_df.index.intersection(atac_rmp.index)
len(shared_index)

4229

In [17]:
dar_gene_df['2mo'] = atac_rmp.iloc[:,0].to_dict()
dar_gene_df['9mo'] = atac_rmp.iloc[:,1].to_dict()
dar_gene_df['18mo'] = atac_rmp.iloc[:,2].to_dict()

In [18]:
non_na_dar_gene_df = dar_gene_df[~dar_gene_df.isna().any(axis=1)]
with_na_dar_gene_df =  dar_gene_df[dar_gene_df.isna().any(axis=1)]

In [19]:
# use PyBigwig to get the atac information for the bw files
bw_dir = "/data/from_luisa/female_atac"
age_list = ['2mo', '9mo', '18mo']
total_counts_df = {}
for age in age_list:
    bw = pyBigWig.open(f"{bw_dir}/Female_{bw_ct}_{age}_merge.bw")
    total_counts = []
    for dmr_id in tqdm(with_na_dar_gene_df.index):
        _chr, start, end = dmr_id.split(':')[0], dmr_id.split(':')[1].split('-')[0], dmr_id.split(':')[1].split('-')[1]
        start, end = int(start), int(end)
        _count = bw.stats(_chr, start, end, type="mean")
        total_counts.append(_count[0])
        
    total_counts_df[age] = total_counts
total_counts_df = pd.DataFrame(total_counts_df)
    

  0%|          | 0/337 [00:00<?, ?it/s]

  0%|          | 1/337 [00:00<00:38,  8.62it/s]

  1%|          | 2/337 [00:00<00:36,  9.12it/s]

  1%|          | 3/337 [00:00<00:39,  8.56it/s]

  1%|          | 4/337 [00:00<00:39,  8.34it/s]

...

 93%|█████████▎| 313/337 [00:09<00:00, 45.33it/s]

 94%|█████████▍| 318/337 [00:09<00:00, 43.22it/s]

 96%|█████████▌| 323/337 [00:09<00:00, 35.56it/s]

 99%|█████████▉| 334/337 [00:09<00:00, 48.70it/s]

100%|██████████| 337/337 [00:09<00:00, 34.79it/s]

In [20]:
for age in age_list:
    with_na_dar_gene_df[age] = total_counts_df[age].values

/var/tmp/ipykernel_77240/436241227.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_na_dar_gene_df[age] = total_counts_df[age].values


In [21]:
with_na_dar_gene_df['log2(18mo/2mo)'] = np.log2(with_na_dar_gene_df['18mo']/with_na_dar_gene_df['2mo'])

/var/tmp/ipykernel_77240/2204217185.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_na_dar_gene_df['log2(18mo/2mo)'] = np.log2(with_na_dar_gene_df['18mo']/with_na_dar_gene_df['2mo'])


In [22]:
# map rows in dar_gene_df that is in with_na_dar_gene_df.index, give 2mo column values based on with_na_dar_gene_df['2mo'].to_dict()
dar_gene_df.loc[with_na_dar_gene_df.index, '2mo'] = with_na_dar_gene_df['2mo'].to_dict()
dar_gene_df.loc[with_na_dar_gene_df.index, '9mo'] = with_na_dar_gene_df['9mo'].to_dict()
dar_gene_df.loc[with_na_dar_gene_df.index, '18mo'] = with_na_dar_gene_df['18mo'].to_dict()

In [23]:
dar_gene_df.to_csv(f'{home_dir}/ml_input/{ct}/{ct}.aDAR_gene.csv',  index=False)

In [24]:
subprocess.run(f"rm {ct}.aDAR_gene.bed", shell=True)

CompletedProcess(args='rm Astro-TE_NN.aDAR_gene.bed', returncode=0)